In [1]:
pip install torcheval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install iterative-stratification

Note: you may need to restart the kernel to use updated packages.


In [3]:
# pip install torch==2.4.1 #+cu121

In [4]:
# pip install xgboost

In [5]:
# pip install catboost

In [6]:
# pip install lightgbm

In [7]:
# from google.colab import drive
import pandas as pd
import numpy as np
import re
import time
import gc
from tqdm.notebook import tqdm
from collections import Counter
import os
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from torcheval.metrics.functional import multiclass_f1_score
from torcheval.metrics import MulticlassF1Score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim as optim
from sklearn.model_selection import StratifiedShuffleSplit
from torcheval.metrics.functional import multiclass_f1_score
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.utils.validation import check_random_state

In [8]:
# drive.mount('./drive')

In [9]:
dataset_dir = '/kaggle/input/lifedatabase/'

# train = pd.read_csv(dataset_dir+'train.csv')
# test = pd.read_csv(dataset_dir+'test.csv')
# sample_submission = pd.read_csv('/kaggle/input/lifedata/sample_submission.csv')
train = pd.read_csv('/kaggle/input/lifedata/train_middle_course.csv')
test_data = pd.read_csv('/kaggle/input/lifedata/test_middle_course.csv')

In [10]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [11]:
CFG = {
    'SEED': 42
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    check_random_state(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# EDA

In [12]:
parts = []
for i in range(4):
     # , map_location=device
  part = torch.load(f'/kaggle/input/lifedata/train_padded_sequences{i+1}.pt').to(torch.float16)
  parts.append(part)
  del part
  gc.collect()

/tmp/ipykernel_30/54422879.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  part = torch.load(f'/kaggle/input/lifedata/train_padded_sequences{i+1}.pt').to(torch.float16)


In [13]:
train_padded_sequences = torch.cat(parts, dim = 1)

del parts
gc.collect()

0

In [14]:
parts = []
for i in range(4):
     # , map_location=device
  part = torch.load(f'/kaggle/input/lifedata/test_padded_sequences{i+1}.pt').to(torch.float16)
  parts.append(part)
  del part
  gc.collect()

In [ ]:
test_padded_sequences = torch.cat(parts, dim = 1)

del parts
gc.collect()

In [ ]:
torch.save(train_padded_sequences, 'train_padded_sequences_full.pt')
torch.save(test_padded_sequences, 'test_padded_sequences_full.pt')

In [15]:
train_padded_sequences.shape#, test_padded_sequences.shape

torch.Size([6201, 4227, 95])

In [16]:
# labels = torch.tensor(train['SUBCLASS'].values, dtype=torch.long)

In [17]:
# labels.unique() # 26 classes

In [20]:
one_hot_encoder = OneHotEncoder(sparse = False) # sparse = False
labels_one_hot = one_hot_encoder.fit_transform(labels.reshape(-1, 1))

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
train['SUBCLASS']

0        KIPAN
1         SARC
2         SKCM
3         KIRC
4       GBMLGG
         ...  
6196      LUAD
6197       LGG
6198      COAD
6199      TGCT
6200      SKCM
Name: SUBCLASS, Length: 6201, dtype: object

In [18]:
le_encoder = LabelEncoder()
labels = le_encoder.fit_transform(train['SUBCLASS'])

In [19]:
labels

array([ 8, 19, 20, ...,  4, 22, 20])

In [ ]:
# flattened_data = train_padded_sequences.view(train_padded_sequences.size(0), -1)

In [ ]:
# labels = np.argmax(labels_one_hot, axis=1)

In [ ]:
# flattened_data.shape, labels.shape

In [21]:
class EmbeddingFeatureExtraction(nn.Module):
    def __init__(self, vocab_size, embedding_size, padding_idx=0):
        super(EmbeddingFeatureExtraction, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=padding_idx)

    def forward(self, x):
        batch_size, feature_dim, sequence_length = x.size()
        x = x.reshape(-1, sequence_length).long()
        x = self.embedding(x)
        x = x.view(batch_size, feature_dim * sequence_length, -1)
        return x

class ConvolutionFeatureExtraction(nn.Module):
    def __init__(self, num_filters = 64, kernel_size = 5, pooling_size = 2):
        super(ConvolutionFeatureExtraction, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(kernel_size, 95), padding=(1, 0))
        self.pool = nn.MaxPool2d(kernel_size=(pooling_size, 1))
        self.dropout = nn.Dropout(0.5)
        self.bn1 = nn.BatchNorm2d(num_filters)
        
    def forward(self, x):
        batch_size, feature_dim, sequence_length = x.size()
        x = x.unsqueeze(1).float()
        x = F.tanh(self.conv(x))
        # x = self.bn1(x)
        x = self.pool(x).squeeze(3)
        # x = self.dropout(x)
        x = x.view(batch_size, -1)
        return x

# class ConvolutionFeatureExtraction(nn.Module):
#     def __init__(self, num_filters = 64, kernel_size = 3, pooling_size = 2):
#         super(ConvolutionFeatureExtraction, self).__init__()
#         self.conv = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(kernel_size, 162), padding=(1, 0))
#         self.pool = nn.MaxPool2d(kernel_size=(pooling_size, 1))
#         self.dropout = nn.Dropout(0.5)
#         self.bn1 = nn.BatchNorm2d(num_filters)
        
#     def forward(self, x):
#         batch_size, feature_dim, sequence_length = x.size()
#         x = x.unsqueeze(1).float()
#         mask = (x != 0).float()
#         x_conv = self.conv(x)
        
#         x_masked = x_conv * mask
        
#         # x_pooled = self.pool(x_masked).squeeze(3)
        
#         x_final = x_masked.view(batch_size, -1)
#         return x_final

# class ConvolutionFeatureExtraction(nn.Module):
#     def __init__(self, num_filters=64, kernel_size=3):
#         super(ConvolutionFeatureExtraction, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(kernel_size, 126), padding=(1, 0))
#         self.pool = nn.MaxPool2d(kernel_size=(2, 1))
#         self.conv2 = nn.Conv2d(in_channels=num_filters, out_channels=num_filters * 2, kernel_size=(kernel_size, 1), padding=(1, 0))
#         self.conv3 = nn.Conv2d(in_channels=num_filters * 2, out_channels=num_filters * 4, kernel_size=(kernel_size, 1), padding=(1, 0))
#         self.dropout = nn.Dropout(0.5)
    
#     def forward(self, x):
#         batch_size, feature_dim, sequence_length = x.size()
#         x = x.unsqueeze(1)
#         x = x.float()

#         x = F.relu(self.conv1(x))
#         x = self.pool(x)
#         x = F.relu(self.conv2(x))
#         x = self.pool(x)
#         x = F.relu(self.conv3(x))
#         x = self.pool(x)

#         x = self.dropout(x)
        
#         x = x.view(batch_size, -1)
#         return x

# class CNN_RNN_Model(nn.Module):
#     def __init__(self, num_filters = 64, kernel_size = 3, pooling_size = 2, hidden_dim = 128, output_dim = 26, num_layers = 1, dropout = 0.5):
#         super(CNN_RNN_Model, self).__init__()
#         self.conv = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(kernel_size, 126), padding=(1, 0))
#         # self.pool = nn.MaxPool2d(kernel_size=(pooling_size, 1))
#         self.rnn = nn.RNN(input_size=37, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
#         self.dropout = nn.Dropout(dropout)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):
#         batch_size, feature_dim, sequence_length = x.size()

#         x = x.unsqueeze(1)
#         x = x.float()

#         x = F.relu(self.conv(x))
#         # x = self.pool(x)
#         print(x.shape)
        
#         # x = x.squeeze(3)
#         x = x.view(batch_size, -1, x.size(3))
#         # height, width = x.shape[2], x.shape[3]
#         # x = x.view(batch_size, width, -1)
#         print(x.shape)
#         # x = x.permute(0, 2, 1)
#         # print(x.shape)
#         # x = x.view(batch_size, sequence_length, -1)

#         output, hidden = self.rnn(x)
        
#         x = output[:, -1, :]
#         x = self.dropout(x)
#         x = self.fc(x) 
#         return x

# class CNN_RNN_Model(nn.Module):
#     def __init__(self, num_filters=64, hidden_dim=128, output_dim=26, num_layers=1, dropout=0.5):
#         super(CNN_RNN_Model, self).__init__()
        
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(3, 126), padding=(1, 0))
#         self.pool1 = nn.MaxPool2d(kernel_size = (2, 1))
#         self.conv2 = nn.Conv2d(in_channels=num_filters, out_channels=num_filters * 2, kernel_size=(3, 64), padding=(1, 0))
#         self.pool2 = nn.MaxPool2d(kernel_size = (2, 1))
#         self.conv3 = nn.Conv2d(in_channels=num_filters * 2, out_channels=num_filters * 4, kernel_size=(3, 32), padding=(1, 0))
#         self.pool3 = nn.MaxPool2d(kernel_size = (2, 1))

#         self.rnn = nn.RNN(input_size=num_filters * 4, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
#         self.dropout = nn.Dropout(dropout)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):
#         batch_size, feature_dim, sequence_length = x.size()
        
#         x = x.unsqueeze(1)
#         x = x.float()

#         x = F.relu(self.conv1(x))
#         x = self.pool1(x)
#         x = F.relu(self.conv2(x))
#         x = self.pool2(x)
#         x = F.relu(self.conv3(x))
#         x = self.pool3(x)

#         x = x.squeeze(3).permute(0, 2, 1)

#         output, hidden = self.rnn(x)

#         x = output[:, -1, :]
#         x = self.dropout(x)
#         x = self.fc(x)
#         # x = F.softmax(x, dim=1)
        
#         return x

# class ConvolutionFeatureExtraction(nn.Module):
#     def __init__(self, num_filters=64, feature_dim=162, kernel_size=3):
#         super(ConvolutionFeatureExtraction, self).__init__()
        
#         self.conv1 = nn.Conv1d(in_channels=feature_dim, out_channels=num_filters, kernel_size=kernel_size, padding=1)
#         self.pool = nn.MaxPool1d(kernel_size=2)
        
#         self.conv2 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters * 2, kernel_size=kernel_size, padding=1)
        
#         self.conv3 = nn.Conv1d(in_channels=num_filters * 2, out_channels=num_filters * 4, kernel_size=kernel_size, padding=1)
        
#         self.dropout = nn.Dropout(0.5)
#     def forward(self, x):
#         batch_size, feature_dim, sequence_length = x.size()
        
#         x = x.view(batch_size, sequence_length, feature_dim)
        
#         x = x.float()

#         x = F.relu(self.conv1(x))
#         x = self.pool(x)
        
#         x = F.relu(self.conv2(x))
#         x = self.pool(x)
        
#         x = F.relu(self.conv3(x))
#         x = self.pool(x)

#         x = self.dropout(x)
        
#         x = x.view(batch_size, -1)
#         return x

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout, hidden_dim, output_dim):
        super().__init__()
        # self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = 0)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        x.permute(0, 2, 1)

        batch_size, sequence_length, column_length = x.size()
        
        # x = self.embedding(x.view(-1, column_length).long())

        x = x.view(batch_size, sequence_length, -1)
        
        output, hidden = self.rnn(x)

        x = output[:, -1, :]

        x = self.dropout(x)

        x = self.fc(x)

        # x = F.softmax(x, dim=1)
        
        return x

In [29]:
class ConvGRUCell(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size):
        super(ConvGRUCell, self).__init__()
        self.hidden_channels = hidden_channels
        self.update_gate = nn.Conv2d(in_channels=input_channels + hidden_channels, out_channels=hidden_channels, kernel_size=kernel_size, padding=(kernel_size[0] // 2, kernel_size[1] // 2))
        self.reset_gate = nn.Conv2d(in_channels=input_channels + hidden_channels, out_channels=hidden_channels, kernel_size=kernel_size, padding=(kernel_size[0] // 2, kernel_size[1] // 2))
        self.out_gate = nn.Conv2d(in_channels=input_channels + hidden_channels, out_channels=hidden_channels, kernel_size=kernel_size, padding=(kernel_size[0] // 2, kernel_size[1] // 2))

    def forward(self, x, hidden_state):
        combined = torch.cat([x, hidden_state], dim=1)
        update = torch.sigmoid(self.update_gate(combined))
        reset = torch.sigmoid(self.reset_gate(combined))
        new_hidden = torch.tanh(self.out_gate(combined) + reset * hidden_state)
        hidden_state = update * hidden_state + (1 - update) * new_hidden
        return hidden_state

class ConvolutionFeatureExtraction(nn.Module):
    def __init__(self, num_filters=64, kernel_size=(3, 3), pooling_size=2, hidden_channels=32):
        super(ConvolutionFeatureExtraction, self).__init__()
        
        self.conv = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(kernel_size[0], 162), padding=(kernel_size[0] // 2, 0))
        self.pool = nn.MaxPool2d(kernel_size=(pooling_size, 1))
        self.dropout = nn.Dropout(0.5)
        self.conv_gru = ConvGRUCell(input_channels=num_filters, hidden_channels=hidden_channels, kernel_size=kernel_size)

    def forward(self, x):
        batch_size, feature_dim, sequence_length = x.size()
        
        x = x.unsqueeze(1)
        x = x.float()

        x = F.relu(self.conv(x))
        x = self.pool(x)

        x = self.dropout(x)

        hidden_state = torch.zeros(batch_size, self.conv_gru.hidden_channels, x.size(2), x.size(3), device=x.device)

        for t in range(x.size(3)):
            hidden_state = self.conv_gru(x[:, :, :, t].unsqueeze(3), hidden_state)

        return hidden_state.view(batch_size, -1)

class ConvGRUCell(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size):
        super(ConvGRUCell, self).__init__()
        self.hidden_channels = hidden_channels
        self.update_gate = nn.Conv2d(in_channels=input_channels + hidden_channels, out_channels=hidden_channels, kernel_size=kernel_size, padding=(kernel_size[0] // 2, kernel_size[1] // 2))
        self.reset_gate = nn.Conv2d(in_channels=input_channels + hidden_channels, out_channels=hidden_channels, kernel_size=kernel_size, padding=(kernel_size[0] // 2, kernel_size[1] // 2))
        self.out_gate = nn.Conv2d(in_channels=input_channels + hidden_channels, out_channels=hidden_channels, kernel_size=kernel_size, padding=(kernel_size[0] // 2, kernel_size[1] // 2))

    def forward(self, x, hidden_state):
        combined = torch.cat([x, hidden_state], dim=1)
        update = torch.sigmoid(self.update_gate(combined))
        reset = torch.sigmoid(self.reset_gate(combined))
        new_hidden = torch.tanh(self.out_gate(combined) + reset * hidden_state)
        hidden_state = update * hidden_state + (1 - update) * new_hidden
        return hidden_state

class ConvolutionFeatureExtraction(nn.Module):
    def __init__(self, num_filters=64, kernel_size=(3, 3), pooling_size=2, hidden_channels=32):
        super(ConvolutionFeatureExtraction, self).__init__()
        
        self.conv = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(kernel_size[0], 162), padding=(kernel_size[0] // 2, 0))
        self.pool = nn.MaxPool2d(kernel_size=(pooling_size, 1))
        self.dropout = nn.Dropout(0.5)
        self.conv_gru = ConvGRUCell(input_channels=num_filters, hidden_channels=hidden_channels, kernel_size=kernel_size)

        # Batch Normalization
        self.bn1 = nn.BatchNorm2d(num_filters)
        self.bn2 = nn.BatchNorm2d(hidden_channels)

    def forward(self, x):
        batch_size, feature_dim, sequence_length = x.size()
        
        x = x.unsqueeze(1)
        x = x.float()

        # Convolution layer with Batch Norm
        x = F.relu(self.bn1(self.conv(x)))
        x = self.pool(x)
        x = self.dropout(x)

        # Initialize hidden state for GRU
        hidden_state = torch.zeros(batch_size, self.conv_gru.hidden_channels, x.size(2), x.size(3), device=x.device)

        # GRU with Batch Norm
        for t in range(x.size(3)):
            hidden_state = self.conv_gru(x[:, :, :, t].unsqueeze(3), hidden_state)

        # Apply Batch Norm to hidden state
        hidden_state = self.bn2(hidden_state)

        return hidden_state.view(batch_size, -1)

In [30]:
train_padded_sequences.shape

torch.Size([6201, 4227, 95])

In [22]:
test_padded_sequences.shape

In [32]:
4230/6

705.0

In [23]:
batch_size = 64
vocab_size = 28
embedding_dim = 128
num_filters = 64
kernel_size = 5
pooling_size = 2

# model = EmbeddingFeatureExtraction(vocab_size, embedding_dim).to('cuda')
model = ConvolutionFeatureExtraction(num_filters = num_filters, kernel_size = kernel_size, pooling_size = pooling_size).to('cuda')

dataset = TensorDataset(train_padded_sequences)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

embedded_sequences = []

with torch.no_grad():
    for batch in data_loader:
        batch = batch[0].to('cuda')

        batch_embedded = model(batch)

        embedded_sequences.append(batch_embedded)

        del batch, batch_embedded
        gc.collect()
        #torch.cuda.empty_cache()

embedded_sequences = torch.cat(embedded_sequences, dim=0)

# test_dataset = TensorDataset(test_padded_sequences)
# test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# test_embedded_sequences = []

# with torch.no_grad():
#     for batch in test_data_loader:
#         batch = batch[0].to('cuda')

#         batch_embedded = model(batch)

#         test_embedded_sequences.append(batch_embedded)

#         del batch, batch_embedded
#         gc.collect()
#         #torch.cuda.empty_cache()

# test_embedded_sequences = torch.cat(test_embedded_sequences, dim=0)

In [24]:
embedded_sequences.shape#, test_embedded_sequences.shape

torch.Size([6201, 135168])

In [25]:
data = embedded_sequences.cpu().numpy()
# test_data = test_embedded_sequences.cpu().numpy()

# data = train_padded_sequences.view(874, -1)

In [26]:
data.shape#, test_data.shape

(6201, 135168)

In [ ]:
pca = PCA()

pca.fit(data)

explained_variance = pca.explained_variance_ratio_

cumulative_variance = np.cumsum(explained_variance)

n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1

print(f'95% 분산을 설명하는 n_components: {n_components_95}')

In [ ]:
n_components_95

In [ ]:
pca = PCA(n_components = 465)
pca = pca.fit(data)
data_pca = pca.transform(data)
# test_data_pca = pca.transform(test_data)

In [52]:
data_pca.shape, test_data_pca.shape

((874, 465), (2546, 465))

In [53]:
def train_epoch(model, dataloader, optimizer, criterion, scheduler, epoch, device, scorer):
    dataset_size = 0
    running_loss = 0.0
    running_score = 0.0
    train_loss = []
    bar = tqdm(enumerate(dataloader), desc='Training', total=len(dataloader), leave=False)
    model.train()
    
    for step, batch in bar:
        optimizer.zero_grad()
        
        data, target = batch
        data = data.float()#.to('cuda')
        target = target.long()#.to('cuda')
        
        output = model(data)
        
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        
        preds = torch.argmax(output, dim=1)
        
        output_np = preds.detach().cpu().numpy()
        target_np = target.detach().cpu().numpy()
        
        score = multiclass_f1_score(target.cpu(), preds.cpu(), average='macro', num_classes=26)
        
        running_loss += loss.item()
        running_score += score
        
        epoch_loss = running_loss / (step + 1)
        epoch_score = running_score / (step + 1)
        
        bar.set_postfix(Train_loss=epoch_loss, Train_score=epoch_score)
        
    return epoch_loss, epoch_score, model

In [54]:
def val_epoch(model, dataloader, criterion, device, scorer):
    outputs = []
    targets = []
    running_loss = 0.0
    bar = tqdm(dataloader, desc='Validating', total=len(dataloader), leave=False)
    model.eval()
    
    with torch.no_grad():
        for batch in bar:
            data, target = batch
            data = data.float()#.to('cuda')
            target = target.long()#.to('cuda')
            
            output = model(data).squeeze()
            
            loss = criterion(output, target)
            
            predicted = torch.argmax(output, dim=1)
            
            outputs.append(predicted.detach().cpu())
            targets.append(target.detach().cpu())
            
            running_loss += loss.item()
    
    outputs = torch.cat(outputs)
    targets = torch.cat(targets)
    
    outputs_np = outputs.numpy()
    targets_np = targets.numpy()
    val_score = multiclass_f1_score(targets.cpu(), outputs.cpu(), average='macro', num_classes=26)
    
    epoch_loss = running_loss / len(dataloader)
    
    return epoch_loss, val_score

In [ ]:
val_scores = []
test_preds = []

reshaped_labels = labels_one_hot
# major_class_labels = reshaped_labels.argmax(axis=1)

splitter = MultilabelStratifiedKFold(n_splits = 5, shuffle = True, random_state = CFG['SEED'])
# splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=CFG['SEED'])

for fold_idx, (train_index, val_index) in enumerate(splitter.split(data_pca, reshaped_labels)):
    
    print(f'Training fold {fold_idx + 1}...')
    
    train_data, val_data = data_pca[train_index], data_pca[val_index]
    train_labels, val_labels = labels[train_index], labels[val_index]

    model = XGBClassifier(tree_method='gpu_hist', gpu_id=0, random_state = CFG['SEED'])
    # model = lgb.LGBMClassifier(boosting_type = 'gbdt', verbose = -1, device = 'gpu', gpu_platform_id = 0, gpu_device_id = 0)
    model.fit(train_data, train_labels) # , callbacks = [lgb.early_stopping(stopping_rounds = 20)], eval_set = [(val_data, val_labels)]

    val_pred = model.predict_proba(val_data)
    # test_pred = model.predict_proba(test_data_pca)

    val_pred = val_pred.argmax(axis = 1)
    # test_pred = test_pred.argmax(axis = 1)

    val_score = f1_score(val_labels, val_pred, average='macro')

    val_scores.append(val_score)
    # test_preds.append(test_pred)

print(f'Validation F1 Scores for each fold: {val_scores}')
print(f'Validation F1 Scores for mean: {np.mean(val_scores)}')

In [99]:
# train_data, val_data, train_labels, val_labels = train_test_split(data_pca, major_class_labels, test_size=0.2, stratify=major_class_labels, random_state=42)

# model = lgb.LGBMClassifier(boosting_type = 'gbdt', verbose = -1, device = 'gpu', gpu_platform_id = 0, gpu_device_id = 0)
# model.fit(train_data, train_labels) # , callbacks = [lgb.early_stopping(stopping_rounds = 20)], eval_set = [(val_data, val_labels)]

# val_pred = model.predict_proba(val_data)

# val_pred = val_pred.argmax(axis = 1)

# val_score = f1_score(val_labels, val_pred, average='macro')

# print(f'Validation F1 Scores: {val_score}')

In [100]:
le_encoder.inverse_transform(test_preds[0])

array(['KIPAN', 'STES', 'PRAD', ..., 'BLCA', 'LAML', 'LAML'], dtype=object)

In [101]:
sample_submission['SUBCLASS'] = le_encoder.inverse_transform(test_preds[0])
sample_submission.to_csv('xg_submission.csv', encoding='UTF-8-sig', index = False)

In [70]:
val_scores = []
num_epochs = 10

reshaped_sequences = train_padded_sequences.mean(dim=2).cpu().numpy()
reshaped_labels = labels_one_hot
major_class_labels = reshaped_labels.argmax(axis=1)

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for fold_idx, (train_index, val_index) in enumerate(splitter.split(reshaped_sequences, major_class_labels)):
    train_data, val_data = train_padded_sequences[train_index], train_padded_sequences[val_index]
    train_labels, val_labels = labels[train_index], labels[val_index]

    train_dataset = TensorDataset(train_data, train_labels)
    val_dataset = TensorDataset(val_data, val_labels)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
    
    print(f'Training fold {fold_idx + 1}...')
    
    patience = 5
    early_stopping_counter = 0
    
    vocab_size = 28
    emb_dim = 128
    sequence_length = 126
    num_classes = 26
    hidden_dim = 128
    dropout = 0.5
    
    # model = CNN_RNN_Model(num_filters=64, kernel_size=3, hidden_dim=128, output_dim=26)#.to(device)
    model = CNN_RNN_Model(num_filters = 50, kernel_size = 3, pooling_size = 2, hidden_dim = 128, output_dim = 26, num_layers = 1, dropout = 0.5).to('cuda')
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
    # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    # scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
    scorer = MulticlassF1Score(average='macro', num_classes=num_classes)
    
    for epoch in tqdm(range(1, num_epochs + 1), desc = 'Epoc'):
        print(time.ctime(), f'Fold {fold_idx + 1}, Epoch {epoch}')
        
        train_loss, train_score, model = train_epoch(model, train_loader, optimizer, criterion, scheduler, device, epoch, scorer)
        
        val_loss, val_score = val_epoch(model, val_loader, criterion, device, scorer)
        
        tqdm.write(f'Epoch {epoch}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val score: {val_score:.4f}')
        val_scores.append(val_score)
        
        if epoch == 1:
            best_loss = val_loss
            best_score = val_score
            
            try:
                torch.save(model.state_dict(), f'best_fold_{fold_idx + 1}.pth')
                print(f'Model saved for fold {fold_idx + 1}')
                
            except Exception as e:
                print(f'Error saving model for fold {fold_idx + 1}: {e}')
                
        if scheduler is not None:
            scheduler.step(val_loss)
            
        if val_loss < best_loss:
            best_loss = val_loss
            best_score = val_score
            early_stopping_counter = 0
            
            try:
                torch.save(model.state_dict(), f'best_fold_{fold_idx + 1}.pth')
                print(f'Model save for fold {fold_idx + 1}')
            
            except Exception as e:
                print(f'Error saving model for {fold_idx + 1}: {e}')
            
        else:
            early_stopping_counter += 1
        
        if early_stopping_counter >= patience:
            print(f'Early stopping triggered. {best_score: .4f}')
            break

Training fold 1...


Epoc:   0%|          | 0/10 [00:00<?, ?it/s]

Sun Oct 20 06:54:55 2024 Fold 1, Epoch 1


Training:   0%|          | 0/6 [00:00<?, ?it/s]

torch.Size([128, 50, 4230, 37])
torch.Size([128, 211500, 37])


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.91 GiB. GPU 0 has a total capacity of 15.89 GiB of which 4.12 GiB is free. Process 2582 has 11.77 GiB memory in use. Of the allocated memory 11.24 GiB is allocated by PyTorch, and 247.33 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [28]:
# val_scores = []
# num_epochs = 10

# mskf = MultilabelStratifiedKFold(n_splits = 2, shuffle = True, random_state = 42)
# reshaped_sequences = train_padded_sequences.mean(dim=2).cpu().numpy()
# reshaped_labels = labels_one_hot

# for fold_idx, (train_index, val_index) in enumerate(mskf.split(reshaped_sequences, reshaped_labels)):
#     train_data, val_data = train_padded_sequences[train_index], train_padded_sequences[val_index]
#     train_labels, val_labels = labels[train_index], labels[val_index]
    
#     train_dataset = TensorDataset(train_data, train_labels)
#     val_dataset = TensorDataset(val_data, val_labels)
    
#     train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
#     val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False)
    
#     print(f'Training fold {fold_idx + 1}...')
    
#     patience = 5
#     early_stopping_counter = 0
    
#     vocab_size = 28
#     emb_dim = 128
#     dropout = 0.2
#     hidden_dim = 128
#     num_classes = 26

#     # model = MutationNet(vocab_size = vocab_size, emb_dim = emb_dim, num_classes = num_classes)
#     # model = mLSTMModel(vocab_size, emb_dim, 128, num_classes)
#     # model = CNN_LSTM_Model(128, 26, num_layers = 1)
#     model = RNNWithConv(162, emb_dim, dropout, hidden_dim, num_classes)
    
#     criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
#     scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
#     scorer = MulticlassF1Score(average='macro', num_classes=26)
    
#     for epoch in tqdm(range(1, num_epochs + 1), desc = 'Epoc'):
#         print(time.ctime(), f'Fold {fold_idx + 1}, Epoch {epoch}')
        
#         train_loss, train_score, model = train_epoch(model, train_loader, optimizer, criterion, scheduler, device, epoch, scorer)
        
#         val_loss, val_score = val_epoch(model, val_loader, criterion, device, scorer)
        
#         tqdm.write(f'Epoch {epoch}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val score: {val_score:.4f}')
#         val_scores.append(val_score)
        
#         if epoch == 1:
#             best_loss = val_loss
#             best_score = val_score
            
#             try:
#                 torch.save(model.state_dict(), f'best_fold_{fold_idx + 1}.pth')
#                 print(f'Model saved for fold {fold_idx + 1}')
                
#             except Exception as e:
#                 print(f'Error saving model for fold {fold_idx + 1}: {e}')
                
#         if scheduler is not None:
#             scheduler.step(val_loss)
            
#         if val_loss < best_loss:
#             best_loss = val_loss
#             best_score = val_score
#             early_stopping_counter = 0
            
#             try:
#                 torch.save(model.state_dict(), f'best_fold_{fold_idx + 1}.pth')
#                 print(f'Model save for fold {fold_idx + 1}')
            
#             except Exception as e:
#                 print(f'Error saving model for {fold_idx + 1}: {e}')
            
#         else:
#             early_stopping_counter += 1
        
#         if early_stopping_counter >= patience:
#             print(f'Early stopping triggered. {best_score: .4f}')
#             break